# UCI Epileptic Seizure Recognition Data Set

In [78]:
from sklearn import svm
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

## Dataset

In [79]:
dataset = pd.read_csv('data/data.csv').replace({'y':[2, 3, 4, 5]}, 0)
X = dataset.iloc[:, 1:179]
y = dataset.iloc[:, 179]

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Model

In [81]:
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

## Test

In [86]:
y_pred = clf.predict(X_test)

In [90]:
accuracy_score(y_test, y_pred, normalize=True)

0.9723320158102767